In [6]:
import pandas as pd
import openpyxl
# packages

# Load and Calculate Inflation Data

In [57]:
def process_exchange_rate_data(file_path, sheet_name, exchange_rate_column, new_column_name):
    # Read the Excel file with the specified header row
    exchange_rate_df = pd.read_excel(file_path, sheet_name=sheet_name)

    # Select the relevant columns and sort by 'Year'
    exchange_rate_df = exchange_rate_df[['Year', exchange_rate_column]]
    exchange_rate_df = exchange_rate_df.sort_values(by='Year', ascending=True)

    # Calculate the percentage change in exchange rate and add it as a new column
    exchange_rate_df[new_column_name] = exchange_rate_df[exchange_rate_column].pct_change()

    return exchange_rate_df

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'data/exchangerates.xlsx'

chf_euro = process_exchange_rate_data(file_path, 'CHF_EUR', 'Average CHF/EUR', 'Exchange Rate Change') # Process CHF/EUR data
chf_usd = process_exchange_rate_data(file_path, 'CHF_USD', 'Average CHF/USD', 'Exchange Rate Change') # Process CHF/USD data)
chf_xbt = process_exchange_rate_data(file_path, 'CHF_XBT', 'Average CHF/XBT', 'Exchange Rate Change') # Process CHF/Bitcoin data)
chf_xau = process_exchange_rate_data(file_path, 'CHF_XAU', 'Average CHF/XAU', 'Exchange Rate Change') # Process CHF/Gold data)
chf_gbp = process_exchange_rate_data(file_path, 'CHF_GBP', 'Average CHF/GBP', 'Exchange Rate Change') # Process CHF/GBP data)
chf_try = process_exchange_rate_data(file_path, 'CHF_TRY', 'Average CHF/TRY', 'Exchange Rate Change') # Process CHF/TRY data)
chf_xag = process_exchange_rate_data(file_path, 'CHF_XAG', 'Average CHF/XAG', 'Exchange Rate Change') # Process CHF/TRY data)
chf_chf = process_exchange_rate_data(file_path, 'CHF_CHF', 'Average CHF/CHF', 'Exchange Rate Change') # Process CHF/CHF data) #K

In [13]:
#############################
#### Calculate Inflatiom ####
#############################

def read_inflation_data(file_path, sheet_name, last_valid_entry, level):
    # Read the Excel file with the specified header row
    data = pd.read_excel(file_path, sheet_name=sheet_name, header=3)
    df = data.iloc[:last_valid_entry + 1]  # This will keep rows up to the last_valid_entry
    if level is not None:
        df = df[df["Level"] == level]  # Select Level in inflation data (default 2)
    else:
        df = df[df["Level"] != 1]  # Select Level in inflation data (default 2)

    return df

def apply_exchange_rate_conversion(df, selected_year, end_year, exchange_rate_df):
    # Select the columns you want to adjust in df
    start_year = selected_year + 1

    # Generate a list of integers from start_year to end_year
    cols_to_adjust = list(range(start_year, end_year + 1))

    # Exchange Rate Conversion Logic
    if exchange_rate_df is not None:
        # Create a dictionary from exchange rate DataFrame
        exchange_rate_change_dict = exchange_rate_df.set_index('Year')['Exchange Rate Change'].to_dict()

        # Adjust the values in df based on the exchange rate change
        for col in cols_to_adjust:
            year = int(col)
            df[col] = df[col] + (exchange_rate_change_dict.get(year, 0) * 100)

    return df

def calculate_cumulative_inflation(df, selected_year):
    # Cumulative Inflation Logic
    # Set the dynamic year values to 100
    df.loc[:, selected_year] = 100

    # Loop through years and calculate real values and subtract 100, starting from the dynamic year
    for year in range(selected_year + 1, 2023):
        df[year] = df[year - 1] * (1 + df[year] / 100)

    for year in range(selected_year, 2023):
        df[year] = df[year] - 100

    subset_columns = ['PosTxt_E', 'PosNo', selected_year] + list(range(selected_year + 1, 2023))
    df = df[subset_columns]
    df = df.rename(columns={'PosNo': 'ID'})

    return df


In [132]:
# Example usage with chf_euro DataFrame
file_path = 'data/su-e-05.02.67.xlsx'
sheet_name = 'VAR_y-1'
last_valid_entry = 415

In [133]:
df_inflation = read_inflation_data(file_path, sheet_name, last_valid_entry,level=None)

In [135]:
Products = list(df_inflation['PosTxt_E'])
Products 

['Food and non-alcoholic beverages',
 'Food',
 'Bread, flour and cereal products',
 'Rice',
 'Flour and other cereals',
 'Bread, pastries and other baked products',
 'Bread',
 'Other bakery products',
 'Small baked goods',
 'Viennese pastries, pastry products',
 'Biscuit/rusk products',
 'Pizza and quiche',
 'Pasta',
 'Breakfast cereals',
 'Other cereal products',
 'Meat, cold cuts and sausages',
 'Meat, fresh or frozen',
 'Beef and veal',
 'Beef',
 'Veal',
 'Pork',
 'Lamb',
 'Poultry',
 'Other meat, fresh',
 'Processed meat and sausages',
 'Sausages',
 'Cold cuts, other meat products and meat preparations',
 'Cold cuts and other meat products',
 'Preparations of raw meat ready to cook',
 'Fish and seafood',
 'Fresh fish',
 'Frozen fish',
 'Tinned fish and smoked fish',
 'Milk, cheese and eggs',
 'Milk and yoghurt',
 'Whole milk',
 'Low fat milk',
 'Yoghurt',
 'Cheese',
 'Hard and semi-hard cheese',
 'Fresh, soft and melted cheese',
 'Other milk products',
 'Drink and milk desserts',
 

In [244]:
import pandas as pd

def apply_exchange_rate_conversion_one_category(df_inflation, selected_product, exchange_rate_df):
    """
    Clean and convert the DataFrame based on the selected product and apply exchange rate conversion.

    Args:
        df_inflation (pd.DataFrame): The input DataFrame.
        selected_product (str): The product to select from the 'PosTxt_E' column.
        columns_to_drop (list): List of column names to drop.
        value_to_drop (str): The value to drop from the DataFrame.
        exchange_rate_df (pd.DataFrame): DataFrame with exchange rate data.

    Returns:
        pd.DataFrame: The cleaned and converted DataFrame.
    """
    # Filter the DataFrame based on the selected product
    filtered_df = df_inflation[df_inflation['PosTxt_E'] == selected_product]

    # Drop specified columns
    filtered_df = filtered_df.drop(columns=['Code', 'COICOP', 'PosType', 'Level', 'Position_D', 'PosTxt_D', 'Position_F', 'PosTxt_F', 'Posizione_I', 'PosTxt_I', 'Item_E', '2023', 1983])

    # Create a boolean mask for columns with the specified value
    mask = (filtered_df == '...').any()

    # Use the boolean mask to drop the columns
    filtered_df = filtered_df.loc[:, ~mask]

    # Check if exchange_rate_df is provided
    if exchange_rate_df is not None:
        # Get the list of columns to adjust
        cols_to_adjust = filtered_df.columns[2:]

        # Generate a list with years where exchange rate is available
        known_exchange_years = list(exchange_rate_df['Year'])

        # Filter cols_to_adjust to only include years that are in known_exchange_years
        cols_to_adjust = [col for col in cols_to_adjust if int(col) in known_exchange_years]
        available_years = cols_to_adjust

        # Create a list of columns to keep, including the 'PosTxt_E' column
        columns_to_keep = ['PosTxt_E','PosNo'] + cols_to_adjust

        # Filter the DataFrame based on the selected columns
        filtered_df = filtered_df[columns_to_keep]

        # Create a dictionary from exchange rate DataFrame
        exchange_rate_change_dict = exchange_rate_df.set_index('Year')['Exchange Rate Change'].to_dict()

        # Adjust the values in df based on the exchange rate change
        for col in cols_to_adjust:
            year = int(col)
            filtered_df[col] = filtered_df[col] + (exchange_rate_change_dict.get(year, 0) * 100)

    return filtered_df, available_years 

# Example usage
selected_product = 'Fees for securities accounts'
# df_inflation is your input DataFrame
# exchange_rate_df is your exchange rate DataFrame
result_df, available_years = apply_exchange_rate_conversion_one_category(df_inflation=df_inflation, selected_product=selected_product,exchange_rate_df=chf_chf)


In [245]:
last_possible_year = available_years[-1]
first_possible_year = available_years[0]
selected_year = first_possible_year -1

In [246]:
last_possible_year 

2022

In [247]:
def calculate_cumulative_inflation(df, selected_year):
    # Cumulative Inflation Logic
    # Set the dynamic year values to 100
    df.loc[:, selected_year] = 100

    # Loop through years and calculate real values and subtract 100, starting from the dynamic year
    for year in range(selected_year + 1, 2023):
        df[year] = df[year - 1] * (1 + df[year] / 100)

    for year in range(selected_year, 2023):
        df[year] = df[year] - 100

    subset_columns = ['PosTxt_E', 'PosNo', selected_year] + list(range(selected_year + 1, 2023))
    df = df[subset_columns]
    df = df.rename(columns={'PosNo': 'ID'})

In [248]:
def calculate_cumulative_inflation_one_category(df, selected_year, last_year):
    # Cumulative Inflation Logic
    # Set the dynamic year values to 100
    df.insert(loc = 2, column = selected_year, value = 100)

    # Loop through years and calculate real values and subtract 100, starting from the dynamic year
    for year in range(selected_year + 1, last_year + 1):
        df[year] = df[year - 1] * (1 + df[year] / 100)

    for year in range(selected_year, last_year + 1):
        df[year] = df[year] - 100

    subset_columns = ['PosTxt_E', 'PosNo', selected_year] + list(range(selected_year + 1, last_year + 1))
    df = df[subset_columns]
    df = df.rename(columns={'PosNo': 'ID'})

    return df

In [249]:
result_df 

,PosTxt_E,PosNo,2017,2018,2019,2020,2021,2022
411,Fees for securities accounts,12536.0,3.4,0.0,1.6,2.4,1.5,2.4


In [250]:
result_df_cumulated = calculate_cumulative_inflation_one_category(df=result_df, selected_year=selected_year,last_year = last_possible_year)

In [251]:
result_df_cumulated

,PosTxt_E,ID,2016,2017,2018,2019,2020,2021,2022
411,Fees for securities accounts,12536.0,0,3.4,3.4,5.0544,7.575706,9.189341,11.809885
